In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
from matplotlib.pyplot import *
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
import seaborn as sns
%matplotlib inline

In [3]:
import xgboost

ImportError: No module named xgboost

In [ ]:
XTrain = pd.read_csv('train.csv')
XTest = pd.read_csv('test.csv')

In [ ]:
XTrain.describe()

In [ ]:
yTrain = XTrain.pop('ACTION')

In [ ]:
model = RandomForestRegressor(n_estimators=100, oob_score=True, n_jobs=-1, random_state=42)
model.fit(XTrain,yTrain)

In [ ]:
y_oob = model.oob_prediction_
print "c-stat: ", roc_auc_score(yTrain, y_oob)

In [ ]:
model.feature_importances_

In [ ]:
feature_importances = pd.Series(model.feature_importances_, index=XTrain.columns)
feature_importances.sort()
feature_importances.plot(kind="barh", figsize=(7,6))

In [ ]:
results = []
n_estimator_options = [500, 1000, 2000, 4000, 6000, 10000]

for trees in n_estimator_options:
    model = RandomForestRegressor(trees, oob_score=True, n_jobs=-1, random_state=42)
    model.fit(XTrain, yTrain)
    print trees, "trees"
    roc = roc_auc_score(yTrain, model.oob_prediction_)
    print "C-stat: ", roc
    results.append(roc)
    print ""
    
pd.Series(results, n_estimator_options).plot();

In [ ]:
features_results = []
max_features_options = ["auto", None, "sqrt", "log2", 0.9, 0.2]

for max_features in max_features_options:
    model = RandomForestRegressor(n_estimators=500, oob_score=True,\
                                 n_jobs=-1, random_state=42,\
                                  max_features=max_features)
    model.fit(XTrain, yTrain)
    print max_features, "option"
    roc = roc_auc_score(yTrain, model.oob_prediction_)
    print "C-stat: ", roc
    features_results.append(roc)
    print ""
    
pd.Series(features_results, max_features_options).plot(kind="barh", xlim=(.85, .88))

In [ ]:
leaf_results = []
min_samples_leaf_options = [1,2,3,4,5,6,7,8,9,10]

for min_samples in min_samples_leaf_options:
    model = RandomForestRegressor(n_estimators=500, oob_score=True, n_jobs=-1,\
                                 random_state=42, max_features="auto",\
                                  min_samples_leaf=min_samples)
    model.fit(XTrain,yTrain)
    print min_samples, "min samples"
    roc = roc_auc_score(yTrain, model.oob_prediction_)
    print "C-Stat: ", roc
    leaf_results.append(roc)
    print""

pd.Series(leaf_results, min_samples_leaf_options).plot()

In [ ]:
model = RandomForestRegressor(n_estimators=6000, oob_score=True, n_jobs=-1, random_state=42,\
                              max_features=.9, min_samples_leaf=3)
model.fit(XTrain,yTrain)

XTest has an 'id' column that XTrain didn't, so it won't fit in the predict function. The next statement pops 'id' into a numpy array.

In [ ]:
yID = XTest.pop("id")

In [ ]:
yTest = model.predict(XTest)

In [ ]:
type(yTest), type(yID)

In [ ]:
yID = yID.as_matrix()

In [ ]:
type(yTest), type(yID)

In [ ]:
submission = pd.Series(yTest, index=yID)

In [ ]:
submission.head(5), type(submission)

In [ ]:
submission.to_csv("submission.csv",index=True,sep=',',\
                  header=True,index_label=('Id','Action'))